In [1]:
print("hello world!")

hello world!


# Importing the Packages


In [2]:
!pip install termcolor
!pip install numpy
!pip install tensorflow
!pip install tensorflow_text
!pip install dlai_grader

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.2/153.2 kB 4.2 MB/s eta 0:00:00ta 0:00:01
  Created wheel for dlai_grader: filename=dlai_grader-1.18.0-py3-none-any.whl size=18532 sha256=9e68bb8d0d553fcfa277fa6ada008ac491d24f3020372183e4435c704ba9f5e9
  Stored in directory: /home/codespace/.cache/pip/wheels/fc/b9/77/0a353071ecfff56316f06e9aaf8dfaa1e77880667884c5555c
Successfully built dlai_grader


In [13]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import traceback
import time
import json
from termcolor import colored
import string
import textwrap
import itertools
import numpy as np
import tensorflow_text as tf_text
import tensorflow as tf

import transformer_utils 
import utils

# Will come in handy later
wrapper = textwrap.TextWrapper(width=70)

# Set random seed
np.random.seed(42)

In [3]:
import w3_unittest

# 1.2 C4 Dataset

In [9]:
'''
load example data stored in 'data/c4-en-10k.jsonl'
each line is a json object. 
strip lines from the example data.
'''
with open('data/c4-en-10k.jsonl') as f:
    examples_jsons = [json.loads(line.strip()) for line in f]

# Printing the examples to see how the data looks like
for i in range(5):
    print(f'Example {i+1}: \n\n{examples_jsons[i]}\n\n')

Example 1: 

{'text': 'Beginners BBQ Class Taking Place in Missoula!\nDo you want to get better at making delicious BBQ? You will have the opportunity, put this on your calendar now. Thursday, September 22nd join World Class BBQ Champion, Tony Balay from Lonestar Smoke Rangers. He will be teaching a beginner level class for everyone who wants to get better with their culinary skills.\nHe will teach you everything you need to know to compete in a KCBS BBQ competition, including techniques, recipes, timelines, meat selection and trimming, plus smoker and fire information.\nThe cost to be in the class is $35 per person, and for spectators it is free. Included in the cost will be either a t-shirt or apron and you will be tasting samples of each meat that is prepared.'}


Example 2: 

{'text': 'Discussion in \'Mac OS X Lion (10.7)\' started by axboi87, Jan 20, 2012.\nI\'ve got a 500gb internal drive and a 240gb SSD.\nWhen trying to restore using disk utility i\'m given the error "Not enough

In [10]:
# extract the text from the examples
natural_language_texts = [example['text'] for example in examples_jsons]

# print the first example text
print(f'First example text:\n\n{natural_language_texts[0]}\n\n')

First example text:

Beginners BBQ Class Taking Place in Missoula!
Do you want to get better at making delicious BBQ? You will have the opportunity, put this on your calendar now. Thursday, September 22nd join World Class BBQ Champion, Tony Balay from Lonestar Smoke Rangers. He will be teaching a beginner level class for everyone who wants to get better with their culinary skills.
He will teach you everything you need to know to compete in a KCBS BBQ competition, including techniques, recipes, timelines, meat selection and trimming, plus smoker and fire information.
The cost to be in the class is $35 per person, and for spectators it is free. Included in the cost will be either a t-shirt or apron and you will be tasting samples of each meat that is prepared.




## 1.4 Decode to Natural Language

In [14]:
'''
create tokenizer by using SentencePieceTokenizer.
- open "./models/sentencepiece.model" file
- create SentencePieceTokenizer object
'''
with open('./models/sentencepiece.model', 'rb') as f:
    pre_trained_tokenizer = f.read()

tokenizer = tf_text.SentencepieceTokenizer(pre_trained_tokenizer, out_type=tf.int32)


Special token:
PAD, EOS = 0, 1
In this tokenizer the string </s> is used as EOS token. By default, the tokenizer does not add the EOS to the end of each sentence, so you need to add it manually when required. Let's verify what id correspond to this token:

In [15]:
eos = tokenizer.string_to_id("</s>").numpy()
print("EOS: " + str(eos))

EOS: 1


In [18]:
# print the encoding of each word in natural_language_texts[2] to see how subwards are tokenized
# print the following format word --> tokenized word
tokenized_text = [(list(tokenizer.tokenize(word).numpy()), word) for word in natural_language_texts[2].split()]

print("Word\t\t-->\tTokenization")
print("-"*40)
for element in tokenized_text:
    print(f"{element[1]:<8}\t-->\t{element[0]}")

Word		-->	Tokenization
----------------------------------------
Foil    	-->	[4452, 173]
plaid   	-->	[30772]
lycra   	-->	[3, 120, 2935]
and     	-->	[11]
spandex 	-->	[8438, 26, 994]
shortall	-->	[710, 1748]
with    	-->	[28]
metallic	-->	[18813]
slinky  	-->	[3, 7, 4907, 63]
insets. 	-->	[16, 2244, 7, 5]
Attached	-->	[28416, 15, 26]
metallic	-->	[18813]
elastic 	-->	[15855]
belt    	-->	[6782]
with    	-->	[28]
O-ring. 	-->	[411, 18, 1007, 5]
Headband	-->	[3642, 3348]
included.	-->	[1285, 5]
Great   	-->	[1651]
hip     	-->	[5436]
hop     	-->	[13652]
or      	-->	[42]
jazz    	-->	[9948]
dance   	-->	[2595]
costume.	-->	[11594, 5]
Made    	-->	[6465]
in      	-->	[16]
the     	-->	[8]
USA.    	-->	[2312, 5]
